In [4]:
import ifcopenshell
import numpy as np
from ifcopenshell import geom
import open3d as o3d
from pc_label_map import color_map, color_map_dict

settings = geom.settings()
settings.set(settings.USE_WORLD_COORDS, True) 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import sys
import os
repo_root = os.path.abspath(os.path.join(os.getcwd(), '.', '.'))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

In [5]:
from pprint import pprint
pprint(color_map_dict)

{0: [[1.0, 0.0, 0.0], 'ceiling'],
 1: [[0.0, 1.0, 0.0], 'floor'],
 2: [[0.0, 0.0, 1.0], 'wall'],
 3: [[1.0, 1.0, 0.0], 'beam'],
 4: [[1.0, 0.0, 1.0], 'column'],
 5: [[0.0, 1.0, 1.0], 'window'],
 6: [[0.5, 0.5, 0.5], 'door'],
 7: [[1.0, 0.5, 0.0], 'table'],
 8: [[0.5, 0.0, 1.0], 'chair'],
 9: [[0.5, 1.0, 0.5], 'sofa'],
 10: [[0.5, 0.5, 1.0], 'bookcase'],
 11: [[1.0, 0.5, 0.5], 'board'],
 12: [[0.0, 0.0, 0.0], 'clutter']}


In [6]:
ifc_label_map = {
  "IfcCovering": [
       ("1125344", 3), # "beam", one frame of ceiling used to complete 13 labels
       0, # "ceiling"
      ], 
  "IfcSlab": [
       ("roof", 0), # "ceiling"
       1, # "floor"
      ], 
  "IfcStair": 1, # "floor"
  "IfcWallStandardCase": 2, # "wall"
  "IfcBeam": 3, # "beam"
  "IfcColumn": 4, # "column"
  "IfcWindow": 5, # "window"
  "IfcDoor": 6, # "door"
  "IfcFurnishingElement": [
      ("table", 7), #'table'
      ("chair", 8), # 'chair'
      ("case", 10), #'bookcase'
      ("cabinet", 11), # 'board'
      12  # Default for unmatched furnishings
      ], #12, # "clutter"
  "IfcBuildingElementProxy": [
      ("plaza", 9), # 'sofa'
      12 # Default for unmatched proxies
      ], #12, # "clutter"
  "IfcDistributionElement": 12, # "clutter"  
}

In [7]:
# Load IFC file
script_dir = os.path.dirname(os.getcwd())
ifc_file_path = os.path.join(script_dir, '.', 'docs', 'smartLab.ifc')

ifc_file = ifcopenshell.open(ifc_file_path) 

# Get all building elements (e.g., walls, doors)
elements = []
for tp in ifc_label_map.keys():
    elements += ifc_file.by_type(tp)

In [8]:
def sample_points_on_mesh(verts, faces, spacing_mm=10):
    """
    Sample points on a mesh with a target spacing (e.g., 10mm between points).
    - verts: Array of shape (N, 3) containing mesh vertices.
    - faces: Array of shape (M, 3) containing triangular face indices.
    - spacing_mm: Desired distance between points in millimeters.
    Returns: Sampled points (shape: [K, 3]).
    """
    # Convert faces to NumPy array if it's a tuple
    if isinstance(faces, tuple):
        faces = np.array(faces)
    
    # Reshape to 2D if needed
    if len(faces.shape) == 1:
        faces = faces.reshape(-1, 3)

    triangles = verts[faces]
    
    # Calculate triangle areas (in mm²)
    vec1 = triangles[:, 1] - triangles[:, 0]
    vec2 = triangles[:, 2] - triangles[:, 0]
    areas = 0.5 * np.linalg.norm(np.cross(vec1, vec2), axis=1)

    # Calculate total surface area and number of points needed
    total_area_mm2 = np.sum(areas)
    points_per_mm2 = 1 / (spacing_mm ** 2)
    num_points = int(total_area_mm2 * points_per_mm2)
    num_points = max(num_points, 1)  # Ensure ≥1 point

    # Sample triangles weighted by their area
    probs = areas / areas.sum()
    sampled_tri_indices = np.random.choice(len(faces), size=num_points, p=probs)
    sampled_tris = triangles[sampled_tri_indices]

    # Barycentric coordinate sampling
    u = np.random.rand(num_points, 1)
    v = np.random.rand(num_points, 1)
    mask = (u + v) > 1
    u[mask] = 1 - u[mask]
    v[mask] = 1 - v[mask]
    w = 1 - (u + v)

    # Compute final points
    sampled_points = (u * sampled_tris[:, 0]) + (v * sampled_tris[:, 1]) + (w * sampled_tris[:, 2])
    return sampled_points

In [9]:
points = []
labels = []

for element in elements:
    if element.Representation is None:
        print(f"Skipping {element.GlobalId}: No representation")
        continue
    # Get geometry
    shape = geom.create_shape(settings, element)
    verts = shape.geometry.verts  # Vertex coordinates (flat list)
    faces = shape.geometry.faces  # Triangular faces (indices)
    
    # Reshape vertices into (N, 3) array
    verts = np.array(verts).reshape(-1, 3)
    
    # Generate points on the mesh surface (see Step 4)
    spacing_mm = 0.01  # Points every 10mm
    sampled_points = sample_points_on_mesh(verts, faces, spacing_mm=spacing_mm)
    points.extend(sampled_points)
    
    # Assign label
    element_type = element.is_a()
    label_spec = ifc_label_map.get(element_type, -1)  # Default to -1 for unknown classes
    label = -1

    # Handle list-based specifications with keyword matching
    if isinstance(label_spec, list):
        default = None
        for item in label_spec:
            if isinstance(item, tuple):
                # Get element name in lowercase (handle potential None values)
                element_name = element.Name.lower()
                if item[0] in element_name:
                    label = item[1]
                    break  # First match wins
            elif isinstance(item, int):
                default = item  # Store potential default value
        
        # Use default if no matches found and default exists
        if label == -1 and default is not None:
            label = default
    elif isinstance(label_spec, int):
        # Direct integer mapping
        label = label_spec

    labels.extend([label] * len(sampled_points))

Skipping 3orDiQyEPBwxO8bTLuym2D: No representation


In [ ]:
def add_noise(points, noise_level=0.01):
    noise = np.random.randn(*points.shape) * noise_level
    return points + noise

noisy_points = add_noise(points)

In [10]:

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(color_map[labels])
o3d.visualization.draw_geometries([pcd])

In [11]:
sim_pc_path = '../docs/smartLab_simulated.ply'
o3d.io.write_point_cloud(sim_pc_path, pcd)

True